# 比例ハザードモデルの解釈
本jupyterファイルでは、比例ハザードモデルによる結果の解釈を行う手法とコードを示す。

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 日本語フォントを設定
font = {'family': 'IPAexGothic'}
mpl.rc('font', **font)

%matplotlib inline

---
## 比例ハザード性
生存時間に対する共変量の影響の推定をあるとき、比例ハザードモデルを適用することができる。
### <font color="blue">ハザード比</font>
その共変量の変化に対する影響を評価するハザード比を定義する。ここで、以下の対数ハザードの差が**<font color="red">時間に依存しないこと（比例ハザード性の仮定）</font>**に注意すること。
$$
\begin{eqnarray} 
\mathrm {HR}\left( t, x = (a,b), \beta\right) & = & \frac {h\left(t,a,\beta\right)}{h\left(t,b,\beta\right)}\\
& = & \mathrm {exp} {\left[ \left\{ \ln { \left[ h_{ 0 }\left( t \right)  \right]  } +a\beta  \right\} -\left\{ \ln { \left[ h_{ 0 }\left( t \right)  \right]  } +b\beta  \right\}  \right] }\\
& = & e^{(a-b)\beta}
\end{eqnarray}
$$

このハザード比は、**<font color="blue">全体の期間にわたる生存状況の比較手段であり</font>**、(3)式から得られたハザード比は、**観察期間中はいつでも**$x=a$の属性の1単位期間あたりの死亡率(イベント発生率)が$x=b$の属性の$e^{(a-b)\beta}$倍であることを意味する。

## 名義尺度共変量
まず、1つの二値または名義尺度共変量の回帰係数の解釈について考える。
共変量が、$a=0,b=1$もしくは$a=+1,b=-1$のような二値変数の値であるとき、ハザード比の推定値は、
$$
\hat {\mathrm {HR}}\left(t,a,b,\hat {\beta}\right) = e^{(a-b)\hat {\beta}} \qquad (5)
$$
となる。ハザード比の$100(1-\alpha)$%信頼区間の区間推定は、
$$
\mathrm {exp} {\left[ (a-b)\hat {\beta} \pm \left| a-b \right| \cdot z_{1-\alpha/2} \cdot \hat {SE}\left(\hat {\beta}\right) \right]} \qquad (6)
$$
となる。

 - $\hat {SE}\left(\beta\right)$: 標準誤差

(5)式のハザード比の形は、もし共変量がそれぞれ$a=0$で$b=1$の場合のイベント発生率が**<font color="blue">観察期間中はいつでも</font>**$e^{(a-b)\hat {\beta}}$倍であることを示す。

### 例
WHAS100に性別(gender)を含む比例ハザードモデルを当てはめ、観察期間中のリスクを調査する。なお、性別(`gender`)は、1=女性で0=男性とコード化されている。

In [11]:
# データセットの読み込み
# Variable      Description                      Codes / Units
# id                 ID Code                            1 - 100
# admitdate  Admission Date               mm/dd/yyyy
# foldate       Follow Up Date                 mm/dd/yyyy
# los              Length of Hospital Stay  Days
# lenfol         Follow Up Time                 Days
# fstat         Vital Satus                        1 = Dead,　0 = Alive
# age           Age at Admission            years
# gender     Gender                              0 = Male, 1= Female
# bmi           Body Mass Index             kg/m^2

WHAS_100_DATASET_PATH = 'dataset/whas100.dat'
COLUMNS = ['admitdate', 'foldate', 'los', 'lenfol', 'fstat', 'age', 'gender', 'bmi']

whas_100_df = pd.read_csv(WHAS_100_DATASET_PATH, sep='\s+', header=None, index_col=0, names=COLUMNS)
whas_100_df.head()

,admitdate,foldate,los,lenfol,fstat,age,gender,bmi
1,03/13/1995,03/19/1995,4,6,1,65,0,31.38134
2,01/14/1995,01/23/1996,5,374,1,88,1,22.65790
3,02/17/1995,10/04/2001,5,2421,1,77,0,27.87892
4,04/07/1995,07/14/1995,9,98,1,81,1,21.47878
5,02/09/1995,05/29/1998,4,1205,1,78,0,30.70601


In [7]:
from lifelines import CoxPHFitter

x_cols = ['gender']
y_cols = ['lenfol', 'fstat']

cox_ph_model = CoxPHFitter()
cox_ph_model.fit(whas_100_df[x_cols + y_cols], duration_col='lenfol', event_col='fstat')
cox_ph_model.summary

,coef,exp(coef),se(coef),z,p,lower 0.95,upper 0.95
gender,0.554815,1.741618,0.282389,1.964721,0.049447,0.001343,1.108286


求めた回帰係数からハザード比の点推定値を算出する。

In [4]:
# ハザード比の追加
HAZARD_RATIO  = {
    'gender': 1
}

for col in HAZARD_RATIO.keys():
    beta = cox_ph_model.hazards_[col].loc['coef']
    print('{col}のハザード比: {HR}'.format(col=col, HR=np.exp(beta)))

genderのハザード比: 1.741618008307074


上記のハザード比から、観察期間をとおして、女性は男性の約1.74倍の割合で死亡(イベントが発生)すると解釈できる。

In [5]:
from scipy.stats import norm

alpha = 0.05
z = norm.ppf(q=1 - alpha/2, loc=0, scale=1)

a = 1
b = 0
beta = cox_ph_model.hazards_['gender'].loc['coef']
se = cox_ph_model._compute_standard_errors()['gender'].loc['se']

print('{col}のハザード比の{p}%信頼区間: {hr}'.format(col='gender', p=1-alpha, hr=(np.exp((a-b)*beta - np.abs(a-b)*z*se), np.exp((a-b)*beta + np.abs(a-b)*z*se))))

genderのハザード比の0.95%信頼区間: (1.0013441347082699, 3.0291616855010557)


上記の結果からハザード比は、観察期間をとおして、女性の死亡率は、男性の死亡率より74%高いといえ、95%の信頼度で、少なくとも0.2%、多ければ203%高いと説明できる。

### 名義尺度共変量が$K$水準(2つ以上)を有する場合

In [12]:
whas_100_df.head()

,admitdate,foldate,los,lenfol,fstat,age,gender,bmi
1,03/13/1995,03/19/1995,4,6,1,65,0,31.38134
2,01/14/1995,01/23/1996,5,374,1,88,1,22.65790
3,02/17/1995,10/04/2001,5,2421,1,77,0,27.87892
4,04/07/1995,07/14/1995,9,98,1,81,1,21.47878
5,02/09/1995,05/29/1998,4,1205,1,78,0,30.70601


## 連続尺度共変量
連続尺度共変量の$c$単位の変化に対応する対数ハザード関数の変化を$a = x + c$および$b = x$として、(2)式および(3)式を用いると、対数ハザード関数の変化は、
$$
\left\{ \ln {h_{0}\left(t\right)} + (x+c)\beta \right\} - \left\{ \ln {h_{0}\left(t\right)} + x\beta \right\} = (x+c)\beta - x\beta\\
= c\beta
$$
となる。ハザード比は、
$$
\hat {\mathrm {HR}}\left(c\right) = e^{c\hat {\beta}}
$$
として推定され、ハザード比の$100(1-\alpha)$%信頼区間の区間推定は、
$$
\mathrm {exp} {\left[ c\hat {\beta} \pm z_{1-\alpha/2} \cdot \left| c\right| \cdot \hat {SE}\left(\hat {\beta}\right) \right]}
$$
である。

### 例
WHAS100を用い、年齢を共変量として、その共変量の単位変化のハザード比を算出する。

In [13]:
from lifelines import CoxPHFitter

x_cols = ['age']
y_cols = ['lenfol', 'fstat']

cox_ph_model = CoxPHFitter()
cox_ph_model.fit(whas_100_df[x_cols + y_cols], duration_col='lenfol', event_col='fstat')
cox_ph_model.summary

,coef,exp(coef),se(coef),z,p,lower 0.95,upper 0.95
age,0.045671,1.04673,0.01195,3.82193,0.000132,0.02225,0.069092


年齢が5歳変化したときのハザード比を算出する。

In [29]:
coef_df = cox_ph_model.summary
coef_df = coef_df.reset_index()
coef_df = coef_df.rename(columns={'index': '変数', 'coef': '回帰係数'})[['変数', '回帰係数']]
coef_df.head()

,変数,回帰係数
0,age,0.045671


In [30]:
# ハザード比の追加
HAZARD_RATIO  = {'age': 5}

In [31]:
def add_point_estimation_of_hazard_ratio(row):
    """
    ハザード比の点推定の値を返す
    """
    variable, beta = row
    if variable in HAZARD_RATIO:
        n_th = HAZARD_RATIO[variable]
    else:
        n_th = 1
    return np.exp(n_th * beta)

In [32]:
coef_df['ハザード比'] = coef_df[['変数', '回帰係数']].apply(add_point_estimation_of_hazard_ratio, axis=1)

In [33]:
from scipy.stats import norm


alpha = 0.05
z = norm.ppf(q=1 - alpha/2, loc=0, scale=1)

beta = cox_ph_model.hazards_['age'].loc['coef']
se = cox_ph_model._compute_standard_errors()['age'].loc['se']
c = HAZARD_RATIO['age']

low = np.exp(c*beta - np.abs(c)*z*se)
high = np.exp(c*beta + np.abs(c)*z*se)

coef_df['ハザード比の区間推定({0})%'.format(1 - alpha)] = '{0:f}, {1:f}'.format(low, high)
coef_df

,変数,回帰係数,ハザード比,ハザード比の区間推定(0.95)%
0,age,0.045671,1.256533,"1.117675, 1.412642"


↑から、年齢が5歳変化した場合

 - ハザード比 : 観察期間をとおして、年齢が5歳増加するごとにハザードは約26%増加し、これはどの年齢に対する増加を計算しても同じである。
 - 区間推定 : 観察期間をとおして、65歳の人の死亡率は、60歳の人の死亡率より26%高いといえ、95%の信頼度で、少なくとも12%、多ければ42%高いと説明できる。

In [25]:
# ハザード比の追加
HAZARD_RATIO  = {
    'Age_in_years': 10,
    "Celltype='large'": 1,
    "Celltype='smallcell'": 1,
    "Celltype='squamous'": 1,
    'Karnofsky_score': 10,
    'Months_from_Diagnosis': 3,
    "Prior_therapy='yes'": 1,
    "Treatment='test'": 1
}

def add_point_estimation_of_hazard_ratio(row):
    """
    ハザード比の点推定の値を返す
    """
    variable, beta = row
    if variable in HAZARD_RATIO:
        n_th = HAZARD_RATIO[variable]
    else:
        n_th = 1
    return np.exp(n_th * beta)


def add_interval_estimation_of_hazard_ratio(row, alpha = 0.05):
    """
    ハザード比の区間推定の値を返す
    """
    return low_estimated_value, high_estimated_value

coef_df['ハザード比'] = coef_df[['変数', '回帰係数']].apply(add_point_estimation_of_hazard_ratio, axis=1)
# coef_df['ハザード比の区間推定]
coef_df

,変数,回帰係数,ハザード比
0,Age_in_years,-0.0178705,0.836352
1,Celltype='large',-0.842767,0.430518
2,Celltype='smallcell',0.00444637,1.004456
3,Celltype='squamous',-1.22398,0.294059
4,Karnofsky_score,-0.0381951,0.682529
5,Months_from_Diagnosis,-0.00415532,0.987611
6,Prior_therapy='yes',-0.164954,0.847933
7,Treatment='test',0.154291,1.166830


## 多変量モデル
リスク因子または治療変数$d$と、他の変数$x$を考える。ここで、共変量$x$は結果変数と有意に関連していると仮定する。

共変量$x$は次のモデルの当てはめを考える。

$x$を含むモデル

 1. 交絡因子: $d$を含むが、$x$は含まないモデル
 2. 関連性の効果修飾因子: $d$と$x$の両方を含むモデル
 3. 交絡因子または効果修飾因子のどちらでもない: $d,x$とそれらの交互作用項$x \times d$を含むモデル

事例の前に、交絡と交互作用に対する概念的理解と視覚的把握を促すため、2変数の設定を考える。

主たるリスク因子$d$は2水準(0=なし、1=あり)であり、主な解析の目的は$d$のハザード比を推定することであると仮定する。<br>

$d$のみを含む比例ハザードモデルは、対数ハザード関数
$$
\ln {h_{0}\left(t\right)} + d \theta_{1}
$$
をもつ。$d$の2水準での対数ハザードの差は、
$$
\left( \ln {h_{0}\left(t\right)} + 1\theta_{1} \right) - \left( \ln {h_{0}\left(t\right)} + 0\theta_{1} \right) = \theta_{1}
$$
となる。

$d$と$x$の両方を含む２番目のモデルにおける対数ハザード関数は、
$$
\ln {h_{0}\left(t\right)} + d \beta_{1} + x\beta_{2}
$$
となる。上式から、調整された対数ハザードの差は、
$$
\left( \ln {h_{0}\left(t\right)} + 1\beta_{1} + x\beta_{2} \right) - \left( \ln {h_{0}\left(t\right)} + 0\beta_{1} + x\beta_{2} \right) = \beta_{1} + (x - x)\beta_{2}\\
= \beta_{1}
$$
となる。

## 調整生存関数の解釈と利用
モデルの推定値の解釈と共変量で調整された生存関数を説明する。
